In [4]:
import pandas as pd
import numpy as np
import re

In [5]:
def rest(df):
    game = np.array(df['G'].fillna(0))
    rested = game == 0
    df['Rest'] = rested*1
    return df

def str_to_int(s):
    if type(s) == int or type(s) == float:
        return s
    if len(re.findall('\d', s)) != 0:
        if '.' in s:
            return float(s)
        return int(s[:2])
    return 0

def min_to_int(s):
    if len(re.findall('\d', s)) != 0:
        return int(s.split(':')[0])
    return 0

def plus_min(s):
    if type(s) == int or type(s) == float:
        return s
    if len(re.findall('\d', s)) == 0:
        return 0
    if s[0] == '-':
        return int(s[1:]) * -1
    if s[0] == '+':
        return int(s[1:]) 
    return 0

def meaner(df, col):
    col_vals = df[col].apply(str_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_min(df, col):
    col_vals = df[col].apply(min_to_int).values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def meaner_int(df, col):
    col_vals = df[col].values
    length = len(col_vals)
    df[col] = np.cumsum(col_vals)/(np.arange(length) + 1)
    return df

def games(df, col):
    game = df[col].fillna(0)
    length = len(game)
    new_game = []
    for i in np.arange(length):
        new_game.append(max(game[:(i + 1)]))
    df[col] = new_game
    return df

def manipulate(df, inj, player):
    df = df.drop(labels=['Tm', 'GS', 'Unnamed: 5', 'Opp', 'PF', 'Unnamed: 7'], axis=1)
    df = rest(df) 
    df['Age'] = df['Age'].apply(str_to_int) 
    
    df['+/-'] = df['+/-'].apply(plus_min) 
    
    df = games(df, 'G')
    df = meaner_min(df, 'MP')
    df = meaner(df, 'FG')
    df = meaner(df, 'FGA')
    df = meaner(df, '3P')
    df = meaner(df, '3PA')
    df = meaner(df, 'FT')
    df = meaner(df, 'FTA')
    
    df['FG%'] = df['FG']/df['FGA']
    df['3P%'] = df['3P']/df['3PA']
    df['FT%'] = df['FT']/df['FTA']
    
    df = meaner(df, 'ORB') 
    df = meaner(df, 'DRB')
    df = meaner(df, 'TRB')
    df = meaner(df, 'AST')
    df = meaner(df, 'STL')
    df = meaner(df, 'BLK')
    df = meaner(df, 'TOV')
    df = meaner(df, 'PTS')
    df = meaner(df, 'GmSc')
    
    df = meaner_int(df, '+/-')
    
    player_i = inj[inj['Relinquised'] == player]
    player_i = player_i[['Date', 'score']]
    
    df = df.merge(player_i, how='left')
    df['score'] = df['score'].fillna(0)
    df['score'] = np.cumsum(df['score'].values)
    df = df.drop('Date', axis=1)
    return df

In [6]:
i = pd.read_csv('injury_with_score.csv')
i.loc[1719, 'Date'] = '2017-02-28'

In [18]:
def conc(files, names):
    zer = pd.read_csv(files[0])
    one = pd.read_csv(files[1])
    two = pd.read_csv(files[2])
    thr = pd.read_csv(files[3])
    fou = pd.read_csv(files[4])
    fiv = pd.read_csv(files[5])
    six = pd.read_csv(files[6])
    sev = pd.read_csv(files[7])
    eig = pd.read_csv(files[8])
    nin = pd.read_csv(files[9])
    
    nzer = manipulate(zer, i, names[0])
    none = manipulate(one, i, names[1])
    ntwo = manipulate(two, i, names[2])
    nthr = manipulate(thr, i, names[3])
    nfou = manipulate(fou, i, names[4])
    nfiv = manipulate(fiv, i, names[5])
    nsix = manipulate(six, i, names[6])
    nsev = manipulate(sev, i, names[7])
    neig = manipulate(eig, i, names[8])
    nnin = manipulate(nin, i, names[9])
    
    team = pd.concat([nzer, none, ntwo, nthr, nfou, 
                     nfiv, nsix, nsev, neig, nnin])
    team.drop('Rk', axis=1, inplace=True)
    team = team.reset_index()
    team.drop('index', axis=1, inplace=True)
    return team

In [32]:
f = ['al_farouq_aminu.csv', 'cj_mccollum.csv', 'damian_lillard.csv',
         'ed_davis.csv', 'evan_turner.csv', 'jusuf_nurkic.csv',
         'maurice_harkless.csv', 'pat_connaughton.csv', 'shabazz_napier.csv',
    'zach_collins.csv']
n = ['Al-Farouq Aminu', 'C.J. McCollum', 'Damian Lillard', 
         'Ed Davis', 'Evan Turner', 'Jusuf Nurkic',
         'Maurice Harkless', 'Pat Connaughton', 'Shabazz Napier', 'Zach Collins']
t = conc(f, n)

In [33]:
t.to_csv(r'C:\Users\milan\Desktop\por17-18.csv', index = None, header=True)

In [31]:
i[i['Relinquised'] == 'Jusuf Nurkic']

,Date,Team,Acquired,Relinquised,injury,severity,score
357,2016-02-23,Nuggets,NaN,Jusuf Nurkic,knee injury,DTD,1
499,2016-03-14,Nuggets,NaN,Jusuf Nurkic,sore left knee,DTD,1
1920,2017-03-31,Blazers,NaN,Jusuf Nurkic,fractured fibula in right leg,out for season,10
1950,2017-04-04,Blazers,NaN,Jusuf Nurkic,fractured fibula in right leg,DTD,1
2091,2017-04-24,Blazers,NaN,Jusuf Nurkic,fractured fibula in right leg,out for season,10
2360,2017-12-08,Blazers,NaN,Jusuf Nurkic,sprained right ankle,DTD,1


In [34]:
set(t['score'])

{0.0, 1.0}

In [35]:
c = pd.read_csv('c.csv')
k = pd.read_csv('k.csv')
l = pd.read_csv('l.csv')
p = pd.read_csv('p.csv')
s = pd.read_csv('s.csv')
w = pd.read_csv('w.csv')

In [37]:
sev_e = pd.concat([c, k, l, p, s, w])
sev_e = sev_e.reset_index()
sev_e.drop('index', axis=1, inplace=True)
sev_e

,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1.0,25,26.000000,1.000000,11.000000,0.090909,0.000000,3.000000,0.000000,0.000000,...,0.000000,2.000000,2.000000,0.000000,1.000000,2.000000,-3.700000,-5.000000,0,0.0
1,2.0,25,29.000000,1.500000,9.000000,0.166667,0.000000,2.500000,0.000000,0.500000,...,1.500000,3.000000,1.500000,0.000000,1.500000,3.500000,-1.250000,15.000000,0,0.0
2,3.0,25,30.666667,3.333333,10.333333,0.322581,0.666667,3.666667,0.181818,0.333333,...,1.000000,2.666667,1.333333,0.000000,1.333333,7.666667,2.333333,15.333333,0,0.0
3,4.0,25,31.500000,3.750000,10.500000,0.357143,1.250000,4.000000,0.312500,1.000000,...,1.000000,2.000000,1.500000,0.000000,1.750000,9.750000,3.450000,10.000000,0,0.0
4,5.0,25,31.800000,4.200000,10.800000,0.388889,2.200000,4.800000,0.458333,1.200000,...,1.600000,1.800000,2.000000,0.000000,1.800000,11.800000,6.420000,8.000000,0,0.0
5,6.0,25,31.000000,3.833333,10.666667,0.359375,2.000000,4.500000,0.444444,1.000000,...,1.666667,2.000000,2.000000,0.000000,1.500000,10.666667,5.866667,5.833333,0,0.0
6,7.0,25,30.142857,4.285714,10.857143,0.394737,2.000000,4.428571,0.451613,1.285714,...,1.428571,2.142857,1.857143,0.000000,1.571429,11.857143,6.985714,6.142857,0,0.0
7,8.0,25,31.125000,4.000000,10.625000,0.376471,1.875000,4.375000,0.428571,1.375000,...,1.625000,2.250000,1.875000,0.000000,1.625000,11.250000,6.425000,5.125000,0,0.0
8,9.0,25,31.111111,4.000000,10.777778,0.371134,1.888889,4.555556,0.414634,1.444444,...,1.666667,2.333333,2.111111,0.111111,1.555556,11.333333,6.866667,3.555556,0,0.0
9,10.0,25,31.600000,4.600000,11.400000,0.403509,2.100000,4.700000,0.446809,1.300000,...,1.700000,2.400000,2.100000,0.100000,1.700000,12.600000,7.920000,1.900000,0,0.0


In [38]:
sev_e.to_csv(r'C:\Users\milan\Desktop\17-18logs.csv', index = None, header=True)

In [39]:
six = pd.read_csv('16-17logs.csv')
sev = pd.read_csv('17-18logs.csv')

In [40]:
six.head()

,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1.0,24,26.000000,4.000000,8.000000,0.500000,0.000000,3.000000,0.000000,0.000000,...,4.0,2.000000,1.000000,0.00,0.000000,8.000000,6.800,4.0,0,0.0
1,2.0,24,27.000000,6.000000,9.500000,0.631579,1.000000,3.500000,0.285714,0.500000,...,3.0,1.500000,0.500000,0.00,1.500000,13.500000,9.200,8.0,0,0.0
2,3.0,24,25.666667,5.333333,9.333333,0.571429,0.666667,2.333333,0.285714,0.333333,...,2.0,1.333333,0.333333,0.00,1.666667,11.666667,6.800,7.0,0,0.0
3,4.0,24,24.000000,4.500000,9.000000,0.500000,0.500000,2.250000,0.222222,0.250000,...,1.5,1.500000,0.250000,0.25,1.500000,9.750000,4.675,6.0,0,0.0
4,5.0,24,23.800000,4.000000,9.000000,0.444444,0.600000,2.600000,0.230769,0.800000,...,1.8,1.200000,0.600000,0.20,1.200000,9.400000,4.500,7.8,0,0.0


In [42]:
sev.head()

,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1.0,25,26.000000,1.000000,11.000000,0.090909,0.000000,3.000000,0.000000,0.000000,...,0.0,2.000000,2.000000,0.0,1.000000,2.000000,-3.700000,-5.000000,0,0.0
1,2.0,25,29.000000,1.500000,9.000000,0.166667,0.000000,2.500000,0.000000,0.500000,...,1.5,3.000000,1.500000,0.0,1.500000,3.500000,-1.250000,15.000000,0,0.0
2,3.0,25,30.666667,3.333333,10.333333,0.322581,0.666667,3.666667,0.181818,0.333333,...,1.0,2.666667,1.333333,0.0,1.333333,7.666667,2.333333,15.333333,0,0.0
3,4.0,25,31.500000,3.750000,10.500000,0.357143,1.250000,4.000000,0.312500,1.000000,...,1.0,2.000000,1.500000,0.0,1.750000,9.750000,3.450000,10.000000,0,0.0
4,5.0,25,31.800000,4.200000,10.800000,0.388889,2.200000,4.800000,0.458333,1.200000,...,1.6,1.800000,2.000000,0.0,1.800000,11.800000,6.420000,8.000000,0,0.0


In [46]:
milan = pd.concat([six, sev])
milan = milan.reset_index()
milan.drop('index', axis=1, inplace=True)
milan

,G,Age,MP,FG,FGA,FG%,3P,3PA,3P%,FT,...,TRB,AST,STL,BLK,TOV,PTS,GmSc,+/-,Rest,score
0,1.0,24,26.000000,4.000000,8.000000,0.500000,0.000000,3.000000,0.000000,0.000000,...,4.000000,2.000000,1.000000,0.000000,0.000000,8.000000,6.800000,4.000000,0,0.0
1,2.0,24,27.000000,6.000000,9.500000,0.631579,1.000000,3.500000,0.285714,0.500000,...,3.000000,1.500000,0.500000,0.000000,1.500000,13.500000,9.200000,8.000000,0,0.0
2,3.0,24,25.666667,5.333333,9.333333,0.571429,0.666667,2.333333,0.285714,0.333333,...,2.000000,1.333333,0.333333,0.000000,1.666667,11.666667,6.800000,7.000000,0,0.0
3,4.0,24,24.000000,4.500000,9.000000,0.500000,0.500000,2.250000,0.222222,0.250000,...,1.500000,1.500000,0.250000,0.250000,1.500000,9.750000,4.675000,6.000000,0,0.0
4,5.0,24,23.800000,4.000000,9.000000,0.444444,0.600000,2.600000,0.230769,0.800000,...,1.800000,1.200000,0.600000,0.200000,1.200000,9.400000,4.500000,7.800000,0,0.0
5,6.0,24,23.333333,3.833333,9.166667,0.418182,0.500000,2.666667,0.187500,0.666667,...,1.666667,1.166667,0.666667,0.333333,1.166667,8.833333,3.933333,6.333333,0,0.0
6,7.0,24,23.285714,4.000000,9.285714,0.430769,0.571429,2.714286,0.210526,0.571429,...,1.714286,1.571429,0.571429,0.285714,1.142857,9.142857,4.514286,5.285714,0,0.0
7,8.0,24,24.000000,3.875000,9.375000,0.413333,0.750000,3.000000,0.250000,0.625000,...,2.000000,2.000000,0.500000,0.250000,1.000000,9.125000,4.837500,5.750000,0,0.0
8,9.0,24,23.222222,3.666667,8.888889,0.412500,0.666667,2.888889,0.230769,0.888889,...,1.888889,1.777778,0.555556,0.222222,0.888889,8.888889,4.611111,3.666667,0,0.0
9,10.0,24,22.500000,3.400000,8.400000,0.404762,0.600000,2.700000,0.222222,0.800000,...,1.700000,2.000000,0.500000,0.200000,0.900000,8.200000,4.130000,3.000000,0,0.0


In [ ]:
sev_e.to_csv(r'C:\Users\milan\Desktop\17-18logs.csv', index = None, header=True)